In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Read the CSV file
df = pd.read_csv("rawV4.csv", header=None, names=["time", "type", "V", "last_time", 'tDelta'])

# Drop the 'type' column
df = df.drop(columns=["type"])
# df['time'] = df['time'] - df['time'].iloc[0]

# df = df.sort_values(by="time").reset_index(drop=True)
# df = df.iloc[:300]
print(df)

           time        V  last_time  tDelta
0      18852412  2469789   18851895     517
1      18852929  1520261   18852412     517
2      18853446   478543   18852929     517
3      18853964  -769540   18853446     518
4      18854481 -1883345   18853964     517
...         ...      ...        ...     ...
28725  33961547 -4747293   33961030     517
28726  33962064 -4001832   33961547     517
28727  33962582 -3190697   33962064     518
28728  33963099 -2245344   33962582     517
28729  33963616 -1326499   33963099     517

[28730 rows x 4 columns]


In [46]:
df['delta'] = df['tDelta']

average_delta = df['delta'].mean()
std_delta = df['delta'].std()
max_delta = df['delta'].max()
min_delta = df['delta'].min()

print(f"Average time delta: {average_delta:.2f}")
print(f"Standard deviation: {std_delta:.2f}")
print(f"Max time delta: {max_delta}")
print(f"Min time delta: {min_delta}")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)

Average time delta: 1495458.76
Standard deviation: 80115654.00
Max time delta: 4294902278
Min time delta: 513
           time        V  last_time      tDelta       delta
0      18852412  2469789   18851895         517         517
1      18852929  1520261   18852412         517         517
2      18853446   478543   18852929         517         517
3      18853964  -769540   18853446         518         518
4      18854481 -1883345   18853964         517         517
5      18854998 -2858455   18854481         517         517
6      18855515 -3727930   18854998         517         517
7      18856032 -4385566   18855515         517         517
8      18856549 -4873839   18856032         517         517
9      18857066 -5202640   18856549         517         517
10     18857583 -5379692   18857066         517         517
11     18858101 -5507723   18857583         518         518
12     18858618 -5497081   18858101         517         517
13     18859135 -4977196   18858618         517   

In [47]:
def compute_negToPos_zero_crossings(t, v):
    crossings = []
    crossings_index = []
    for i in range(len(v) - 1):
        if v[i] < 0 and v[i+1] >= 0:  # Positive-going
            # Linear interpolation to estimate crossing time
            slope = v[i+1] - v[i]
            if slope == 0:
                continue  # avoid divide by zero
            t_cross = t[i] - v[i] * (t[i+1] - t[i]) / slope
            crossings.append(t_cross)
            crossings_index.append(i) # no linear interpolation
            #####
            ##
    return [np.array(crossings), np.array(crossings_index)]

def generate_CT(last_cycle_df, cur_df, last_zc_time, cur_zc_time):

    # Extract one full voltage cycle
    last_cycle_times = last_cycle_df['time'].values
    last_cycle_values = last_cycle_df['V'].values

    # Normalize time to start from 0
    last_cycle_times = last_cycle_times - last_zc_time
    total_duration = last_cycle_times[-1]

    # Normalize the voltage waveform relative to peak
    v_peak = np.abs(last_cycle_values).max()
    if v_peak == 0:
        raise ValueError("v_peak is zero — cannot scale")

    normalized_V = last_cycle_values / v_peak  # -1 to +1 range

    # Interpolation function to get value at any time offset
    interp_fn = np.interp

    CT_output = []

    for _, row in cur_df.iterrows():
        timeD = row['time'] - cur_zc_time

        if 0 <= timeD <= total_duration:
            # Interpolate the waveform shape at this time offset
            v_norm = interp_fn(timeD, last_cycle_times, normalized_V)

            # Scale as needed — for example CT_peak output is 0.6V (adjust as needed)
            CT_amplitude = v_peak # peak current signal voltage
            ct_value = v_norm * CT_amplitude
        else:
            ct_value = 0.0  # outside one cycle

        CT_output.append(ct_value)

    # Add new column to current df
    cur_df = cur_df.copy()
    cur_df['I'] = CT_output
    return cur_df

In [ ]:
zc_times, zcs = compute_negToPos_zero_crossings(df['time'].to_numpy(), df['V'].to_numpy())

last_cycle = pd.DataFrame([])

for i in range(3):
    # print(f'--10cycleChunk--{i//10}')
    start, end = zcs[i], zcs[i+1] # samples before 1st zc, 11th zc
    # print(start + 1,end)
    tempdf = df.iloc[start + 1: end + 2]  #sample after 1st zc : sample after zc
    

    if last_cycle.empty == False:
        # print(zc_times[i - 1], zc_times[i])
        # print(last_cycle)
        # print(tempdf)
        ct_df = generate_CT(last_cycle, tempdf, zc_times[i - 1], zc_times[i])
        print(ct_df)

        # Plot voltage and CT current on the same plot
        plt.figure(figsize=(10, 4))

        plt.plot(ct_df['time'], ct_df['I'], label='Synthesized CT Current (I)', color='orange')
        if 'V' in ct_df.columns:
            plt.plot(ct_df['time'], ct_df['V'], label='Voltage (V)', color='blue', alpha=0.5)

        plt.xlabel('Time (µs)')
        plt.ylabel('Signal')
        plt.title('Simulated CT Output vs Voltage')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        



    last_cycle = tempdf





# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(df[df['type'] == 'V'])
